In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
from new_solutions.simple_problem import study_many_problems
from runner.svg2plan import run_new_layout
from new_solutions.selection import *
from itertools import accumulate
from operator import add 
from functools import reduce
from visuals.plotter import plot_general
# from unit_tests.test_solutions import *
from typing import Literal, List
from enum import Enum
from collections import Counter
from typing import TypeVar
from typing import Any
from domains.domain import Domain

import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
report = run_new_layout()
c = Cook(report)


executer> 
 begining to execute stacking
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one probl

In [5]:
c.bl.problems

[Problem(index=1, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'dining'),
 Problem(index=2, problem_type=ProblemType.HOLE, resolved=True, nbs=['transit_space', 'laundry', 'm_bath', 'bath'],
 Problem(index=3, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_closet', 'dining'),
 Problem(index=4, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'bath')]

In [7]:
c.run_again()

running again -> 1
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one problem
many problem
one pr

In [9]:
c.plot(1)

In [10]:
c.bl.problems

[Problem(index=1, problem_type=ProblemType.OVERLAP, resolved=True, nbs=('m_bedroom', 'dining'),
 Problem(index=3, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_closet', 'dining'),
 Problem(index=4, problem_type=ProblemType.OVERLAP, resolved=True, nbs=('m_bedroom', 'bath')]

In [8]:
c.bl_hist[1].problems

[Problem(index=1, problem_type=ProblemType.OVERLAP, resolved=True, nbs=('m_bedroom', 'dining'),
 Problem(index=3, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_closet', 'dining'),
 Problem(index=4, problem_type=ProblemType.OVERLAP, resolved=True, nbs=('m_bedroom', 'bath')]

In [14]:
c.run_again()

running again -> 4


IndexError: list index out of range

In [12]:
c.plot(2)

In [13]:
c.run_again()

running again -> 3


IndexError: list index out of range

In [6]:
c.run_again()

running again -> 3
reporter> Couldn't identify problems.. 


AttributeError: 'SideHoleIdentifier' object has no attribute 'problems'

In [ ]:
c.plot(4)